<a href="https://colab.research.google.com/github/7Pam2015-group-11/spaceship_titanic/blob/main/spaceshiptatanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import StackingClassifier
import pandas as pd_spaceship
import numpy as np_spaceship
import seaborn as sns_spaceship
import matplotlib.pyplot as plt_spaceship
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier


from google.colab import drive
drive.mount('/content/drive/')

# Load data
train_df_spaceship = pd_spaceship.read_csv('/content/drive/MyDrive/train.csv')
test_df_spaceship = pd_spaceship.read_csv('/content/drive/MyDrive/test.csv')




MessageError: Error: credential propagation was unsuccessful

In [ ]:
#Display the first few rows of the training dataset to understand its structure
train_df_spaceship.head()

# Extract and store the 'PassengerId' column from the test dataset for submission purposes
test_passenger_ids = test_df_spaceship['PassengerId']

# Print the number of missing values for each column in the training dataset
print(train_df_spaceship.isnull().sum())

# Plot the distribution of the target variable 'Transported' to visualize the class balance
sns_spaceship.countplot(data=train_df_spaceship, x='Transported', color='skyblue')
plt_spaceship.show()

# Create histograms for the numerical features to examine their distributions
numeric_features = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
train_df_spaceship[numeric_features].hist(bins=15, figsize=(15, 10), color='skyblue', edgecolor='black')
plt_spaceship.show()

In [ ]:
# Remove the 'PassengerId', 'Name', 'Cabin', and 'VIP' columns from both the training and test datasets to clean the data
train_df_spaceship.drop(['PassengerId', 'Name', 'Cabin', 'VIP'], axis=1, inplace=True)
test_df_spaceship.drop(['PassengerId', 'Name', 'Cabin', 'VIP'], axis=1, inplace=True)


In [ ]:
# Specify the categorical and numerical features to be processed separately
categorical_features_spaceship = ['HomePlanet', 'CryoSleep', 'Destination']
numeric_features_spaceship = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Create imputers for filling missing values in numerical and categorical features
numeric_imputer = SimpleImputer(strategy='median')
categorical_imputer = SimpleImputer(strategy='most_frequent')

# Fill missing values in numerical features using the median value of each column
train_df_spaceship[numeric_features_spaceship] = numeric_imputer.fit_transform(train_df_spaceship[numeric_features_spaceship])
test_df_spaceship[numeric_features_spaceship] = numeric_imputer.transform(test_df_spaceship[numeric_features_spaceship])

In [ ]:
# Fill missing values in categorical features using the most frequent value of each column
train_df_spaceship[categorical_features_spaceship] = categorical_imputer.fit_transform(train_df_spaceship[categorical_features_spaceship])
test_df_spaceship[categorical_features_spaceship] = categorical_imputer.transform(test_df_spaceship[categorical_features_spaceship])

# Convert categorical variables into numerical values using LabelEncoder for model compatibility
label_encoders = {}
for feature in categorical_features_spaceship:
    label_encoders[feature] = LabelEncoder()
    train_df_spaceship[feature] = label_encoders[feature].fit_transform(train_df_spaceship[feature])
    test_df_spaceship[feature] = label_encoders[feature].transform(test_df_spaceship[feature])

In [ ]:
# Feature scaling
scaler = StandardScaler()
train_df_spaceship[numeric_features_spaceship] = scaler.fit_transform(train_df_spaceship[numeric_features_spaceship])
test_df_spaceship[numeric_features_spaceship] = scaler.transform(test_df_spaceship[numeric_features_spaceship])

In [ ]:
# Splitting the data
X = train_df_spaceship.drop('Transported', axis=1)
y = train_df_spaceship['Transported']
